# Adler wedge photometry
Adler provides a method of running wedge photometry on the cutout image of a detection.
To do this we invoke the `astscript-radial-profile` command line tool from [gnuastro](https://www.gnu.org/software/gnuastro/manual/html_node/Invoking-astscript_002dradial_002dprofile.html), using python subprocess.
`astscript-radial-profile` calculates various statistics (sum, mean, median, std) for pixels in radial bins (starting from the centre of the image to the edge) in a given azimuthal range.
We therefore call `astscript-radial-profile` for a number of azimuthal bins covering the full 0-360 degree range, and use this to determine the flux contained in each "wedge".

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy.io import fits
from astropy import visualization as aviz
from photutils.aperture import CircularAnnulus, CircularAperture

from adler.science.WedgePhot import WedgePhot

In [ ]:
# Define the fits image (and hdu index) to be analysed
# Cutouts were retrieved from ZTF: https://irsa.ipac.caltech.edu/applications/ztf/
infits = "../tests/data/ztf_Didymos-system-barycenter20065803_20221201402616_000567_zr_c10_o_q1_scimrefdiffimg.fits.fz"  # faint tail by eye
inhdu = 1

In [ ]:
# Number of azimuthal bins to perform wedge photometry over
N_wedge = 10

# Define inner and outer radii to calculate flux statistics over
rin = 10  # pixels
rout = 25  # pixels

In [ ]:
# Inspect the fits image header data unit
hdu = fits.open(infits)
hdu.info()

In [ ]:
# Get image data and header
img = hdu[inhdu].data
hdr = hdu[inhdu].header

In [ ]:
# Look at the image
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

norm = aviz.ImageNormalize(img, interval=aviz.ZScaleInterval())
s1 = ax1.imshow(img, norm=norm, origin="lower")
cbar = plt.colorbar(s1)

plt.show()

In [ ]:
# Set up the adler wedge photometry class
wp = WedgePhot(
    fits_file=infits,  # image to analyse
    i_hdu=inhdu,  # hdu index of image
    N_wedge=N_wedge,  # number of wedges
    measure="sum,mean,median,sigclip-mean,sigclip-std",  # statistics to calculate
)
wp

In [ ]:
# These are the azimuthal bin edges
wp.az

In [ ]:
# This function calculates the flux statistics for each bin and compiles the results.
# It constructs the relevant astscript-radial-profile commands and calls them with subprocess
# The results table for each wedge is written to a temporary file, which is then read into a pandas DataFrame
# when running in jupyter, we need to make subprocess activate conda and run in the correct environment
wp_results = wp.run_wedge_phot(keep_files=True)
# wp_results = wp.run_wedge_phot(
#     conda_start=". /opt/lsst/software/stack/conda/etc/profile.d/conda.sh", conda_env="adler-dev-gnu"
# )

In [ ]:
# the results are stored in a dictionary, keyed by the index of each wedge
# each key contains the upper and lower azimuthal bin edges of each wedge (az_min, az_max)
# and the pandas DataFrame of the radial statistics (data)
wp_results

In [ ]:
# plot the total flux in each radial bin, for each wedge
x_plot = "RADIUS"
y_plot = "SUM"

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

for i in wp_results.keys():
    df_plot = wp_results[i]["data"]
    ax1.plot(df_plot[x_plot], df_plot[y_plot], label=i)
    _df_plot = df_plot[~np.isnan(df_plot[y_plot])]
    ax1.scatter(
        _df_plot.iloc[-1][x_plot],
        _df_plot.iloc[-1][y_plot],
        c="k",
    )


ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.legend()

plt.show()

In [ ]:
# represent this as a radial plot showing the total flux in each wedge
r_plot = "SUM"

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0], projection="polar")

# loop over each wedge
for i in wp_results.keys():

    amin = wp_results[i]["az_min"]
    amax = wp_results[i]["az_max"]
    a_plot = np.radians((amin + amax) * 0.5)
    df_plot = wp_results[i]["data"]
    df_plot = df_plot[(df_plot["RADIUS"] >= rin) & (df_plot["RADIUS"] < rout)]
    df_plot = df_plot.dropna(how="any")
    df_plot = df_plot[
        (df_plot[r_plot] != 0) & (df_plot[r_plot] > 0)
    ]  # negative values of flux can make the bins weird
    ax1.scatter(a_plot, np.sum(df_plot[r_plot]), c="k")
    ax1.plot([0, a_plot], [0, np.sum(df_plot[r_plot])], c="k")

ax1.scatter(0, 0, c="r", marker="x")

ax1.set_xticks(np.radians(wp.az))

ax1.set_rlabel_position(0)  # Move radial labels away from plotted line
ax1.set_rticks(ax1.get_yticks()[::2])  # Less radial ticks

fig.suptitle("{}(flux)".format(r_plot))

plt.show()

In [ ]:
# Function for finding points on a circle
def circle_xy(t, r=1, a=0, b=0):
    """
    t: float
        Azimuthal angle, radians
    r: float
        Radius of the circle
    a: float
        X coordinate of circle centre
    b: float
        Y coordinate of circle centre
    """
    x = a + (r * np.cos(t))
    y = b + (r * np.sin(t))
    return x, y

In [ ]:
plot_img = img

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

# Display the image
norm = aviz.ImageNormalize(plot_img, interval=aviz.ZScaleInterval())
s1 = ax1.imshow(plot_img, norm=norm, origin="lower")
c1 = plt.colorbar(s1)

# https://photutils.readthedocs.io/en/latest/user_guide/aperture.html#local-background-subtraction

# Define the centre of the image
positions = [tuple(np.array(plot_img.shape) / 2)]

# Determine and plot the outer most radius used by WedgePhot, either the image edge or defined by user
if wp.ap_rad_out is None:
    ap_rad_out = np.amin(plot_img.shape) / 2.0
else:
    ap_rad_out = wp.ap_rad_out
aperture = CircularAperture(positions, r=ap_rad_out)
ap_patches = aperture.plot(color="red", lw=2, label="outer aperture")

# Plot the annulus over which the statistics are calculated
annulus_aperture = CircularAnnulus(positions, r_in=rin, r_out=rout)
ann_patches = annulus_aperture.plot(color="r", lw=2, label="Background annulus")

# Plot the wedges
x_in, y_in = circle_xy(np.radians(wp.az), r=rin, a=positions[0][0], b=positions[0][0])
x_out, y_out = circle_xy(np.radians(wp.az), r=rout, a=positions[0][0], b=positions[0][0])
for i in range(len(wp.az)):
    ax1.plot([x_in[i], x_out[i]], [y_in[i], y_out[i]], "r")

fig.suptitle(infits)

plt.show()